In [1]:
import pandas as pd
import os
import glob
import numpy as np

# Match Smart Sheets to Power BI

In [2]:
# #DRR21048 - Registered Professional Forester Viewer
path1 = r"C:\Users\jacque.trahan\Downloads\Tree*"
file_type1 = ".csv"
load1 = glob.glob(path1 + file_type1)
df1 = pd.read_csv(max(load1, key=os.path.getctime))

In [3]:
tree = df1.copy()

In [4]:
# tree = tree[tree['County'].isin(['LAKE'])]

In [5]:
tree = tree[tree['County'].isin(['EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'ALPINE'])]

In [6]:
path2 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
file_type2 = ".xlsx"
load2 = glob.glob(path2 + file_type2)
df2 = pd.read_excel(max(load2, key=os.path.getctime))

In [14]:
ss = df2.copy()
ss = ss[['APN_ROW Segment','Street #', 'Street Name', 'County', 'Haz Trees Assessment', 'WLPZ Zone?', 'In lieu practices?']]

# compare Registered Professional Forester Viewer

In [15]:
def compare(df, comp1, comp2):
    if df[comp1] == df[comp2]:
        return "Match"
    else:
        return "No Match"

In [16]:
final = ss.merge(tree,
                left_on='APN_ROW Segment',
                right_on='APN',
                how='outer',
                suffixes=('_SS', '_Tree'))

In [17]:
final = final[['APN_ROW Segment','WLPZ Zone?_SS', 'WLPZ Zone?_Tree', "In lieu practices?_SS", "In lieu practices?_Tree", 'County_SS' ]]

In [18]:
final.insert(final.columns.get_loc('WLPZ Zone?_Tree')+1, "WLPZ_Match",
            final.apply(compare, comp1='WLPZ Zone?_SS', comp2='WLPZ Zone?_Tree', axis=1))

In [19]:
final.insert(final.columns.get_loc("In lieu practices?_Tree")+1, 'In lieu practices?_Match',
            final.apply(compare, comp1="In lieu practices?_SS", comp2="In lieu practices?_Tree", axis=1))

In [20]:
final_style = final.style.set_properties(**{'background-color': 'yellow'},
                                        subset=["WLPZ_Match", 'In lieu practices?_Match'])

In [21]:
final_style.to_excel('DRR21048 - Registered Professional Forester Viewer - Central.xlsx')

# PioParedes, Agustin
tree counts

In [9]:
# Agustin = df2.copy()
# Agustin.groupby(['County'])['Number of Trees'].count()

In [10]:
# Agustin.groupby(['County'])['Number of Trees'].sum()

In [15]:
# final = final[(final['WLPZ Zone?_SS'].isnull()) & (final['Haz Trees Assessment'].notnull())]

In [16]:
# final.fillna({'WLPZ Zone?_SS': "0",
#              'WLPZ Zone?_Tree': "0",
#              "In lieu practices?_SS":"0",
#               "In lieu practices?_Tree": "0"}, inplace=True)

# WLPZ zones assessed and whats left

### Smart sheets only for now

In [21]:
# df2.info(verbose=True)

### Central

In [22]:
ss_c = df2.copy()
ss_c = ss_c[(ss_c['Withdrawal'].isna()) & (ss_c['Not Eligible'].isna())]
ss_c = ss_c[['APN_ROW Segment','Street #', 'Street Name', 'County', 'WLPZ Zone?','Division', 'Withdrawal', 'Not Eligible']]
WLPZ_done_c = ss_c[(ss_c['WLPZ Zone?'].notna())]
WLPZ_NotDone_c = ss_c[ss_c['WLPZ Zone?'].isna()]
WLPZ_NotDone_c['WLPZ Zone?'].fillna(1, inplace=True)


C:\Users\jacque.trahan\AppData\Roaming\Python\Python39\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [23]:
final = WLPZ_done_c.merge(WLPZ_NotDone_c,
                       left_on="APN_ROW Segment",
                       right_on="APN_ROW Segment",
                        how="outer",
                       suffixes=("_Done", "_Not Done")).to_excel("WLPZ Summary Central Dixie.xlsx")

###  Northern 

In [24]:
path3 = r"C:\Users\jacque.trahan\Downloads\BOP_North Div 2021 Debris Removal Tracker*"
file_type3 = ".xlsx"
load3 = glob.glob(path3 + file_type3)
df3 = pd.read_excel(max(load3, key=os.path.getctime))

In [25]:
ss_n = df3.copy()
ss_n = ss_n[['APN_ROW Segment','Street #', 'Street Name', 'County', 'WLPZ Zone?','Division','Withdrawal', 'Not Eligible']]
WLPZ_done_n = ss_n[ss_n['WLPZ Zone?'].notna() & (ss_n['Withdrawal'].isna()) & (ss_n['Not Eligible'].isna())]
WLPZ_NotDone_n = ss_n[ss_n['WLPZ Zone?'].isna() & (ss_n['Withdrawal'].isna()) & (ss_n['Not Eligible'].isna())]
WLPZ_NotDone_n['WLPZ Zone?'].fillna(1, inplace=True)


In [26]:
final = WLPZ_done_n.merge(WLPZ_NotDone_n,
                       left_on="APN_ROW Segment",
                       right_on="APN_ROW Segment",
                        how="outer",
                       suffixes=("_Done", "_Not Done")).to_excel("WLPZ Summary-Northern Dixie.xlsx")